In [2]:
%matplotlib inline
import scanpy as sc
import scvi
import matplotlib.pyplot as plt
import pandas as pd
from sctools import integrate, plot, io

Global seed set to 0


# Annotate celltypes with celltypist

In [3]:
adata = sc.read_h5ad(
    '../data/skin_inflammatory_disease.full.integrated.clustered.h5ad'
)
adata

AnnData object with n_obs × n_vars = 332546 × 4000
    obs: 'sample_id', 'patient_id', 'status', 'tissue', 'cell_fraction', 'doublet', 'doublet_score', 'nFeature_RNA', 'nCount_RNA', 'percent_mt', 'percent_ribo', 'qc_pass', '_scvi_batch', '_scvi_labels', 'leiden_scvi_0.1', 'coarse_cell_types'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'coarse_cell_types_colors', 'hvg', 'leiden', 'leiden_scvi_0.1_colors', 'neighbors', 'umap'
    obsm: 'X_scvi', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [ ]:
import celltypist as ct
model = ct.models.Model.load(model = 'Adult_Human_Skin.pkl')
model

In [ ]:
sc.pp.normalize_total(
    adata,
    target_sum = 1e4
)
sc.pp.log1p(adata)
predictions = ct.annotate(
    adata, 
    model = 'Adult_Human_Skin.pkl', 
    majority_voting = True,
    over_clustering = 'leiden_scvi_0.1'
)

In [ ]:
bdata = predictions.to_adata()
sc.pl.umap(
    bdata,
    color = 'majority_voting'
)

In [4]:
adata.write('../data/inflammatory_disease.skin.coarse.h5ad')